In [20]:
import requests
import json

def test_local_server():
    """Test local server trước khi deploy"""
    base_url = "http://localhost:8080"  # Local server
    
    # Test health
    print("=== TESTING LOCAL SERVER ===")
    try:
        health_response = requests.get(f"{base_url}/health", timeout=5)
        print(f"Health: {health_response.status_code} - {health_response.json()}")
    except Exception as e:
        print(f"❌ Local server not running: {e}")
        print("Chạy: npm run dev")
        return
    
    # Test process document
    test_data = {
        "recordId": "recv6dq3ft",
        "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
        "tableID": "tblpzAwC8Ol4VWmZ",
        "idFieldName": "ID",
        "loaiVanBanFieldName": "Loại văn bản - con",
        "hoSoDinhKemFieldName": "Tài liệu đính kèm",
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    try:
        response = requests.post(f"{base_url}/process-document", 
            json=test_data, 
            timeout=30
        )
        
        print(f"\nProcess Document Status: {response.status_code}")
        print(f"Response: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")
        
    except Exception as e:
        print(f"❌ Local test failed: {e}")

if __name__ == "__main__":
    test_local_server()


=== TESTING LOCAL SERVER ===
Health: 200 - {'status': 'OK', 'timestamp': '2025-06-26T09:34:37.826Z'}

Process Document Status: 200
Response: {
  "success": true,
  "instanceCode": "84385131-2981-406E-ADAB-8766E0D9C624",
  "message": "Đã tạo thành công đơn phê duyệt văn bản với 1 tài liệu đính kèm",
  "documentInfo": {
    "loaiVanBan": "Công văn nội bộ"
  },
  "uploadedCodes": [
    "6FA7AAC5-699A-426D-B2DB-C3538B22862A"
  ],
  "documentCount": 1
}


In [ ]:
import requests
import json

def test_cloud_server():
    """Test cloud server trên Google Cloud Run"""
    base_url = "https://document-approval-processor-858039461446.asia-southeast1.run.app"
    
    # Test health
    print("=== TESTING CLOUD SERVER ===")
    try:
        health_response = requests.get(f"{base_url}/health", timeout=10)
        print(f"Health: {health_response.status_code} - {health_response.json()}")
    except Exception as e:
        print(f"❌ Cloud server error: {e}")
        return
    
    # Test debug approval
    print("\n=== TESTING DEBUG APPROVAL ===")
    try:
        debug_response = requests.get(
            f"{base_url}/debug-approval/80979F0E-15E1-479E-8865-DC291883E5ED", 
            timeout=15
        )
        print(f"Debug Approval Status: {debug_response.status_code}")
        if debug_response.status_code == 200:
            print("✓ Debug approval endpoint working")
        else:
            print(f"Debug response: {debug_response.text}")
    except Exception as e:
        print(f"❌ Debug approval failed: {e}")
    
    # Test process document
    print("\n=== TESTING PROCESS DOCUMENT ===")
    test_data = {
        "recordId": "recuPhKBve1BJs",
        "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
        "tableID": "tblpzAwC8Ol4VWmZ",
        "idFieldName": "ID",
        "loaiVanBanFieldName": "Loại văn bản - con",
        "hoSoDinhKemFieldName": "Tài liệu đính kèm",
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    try:
        print("Sending request to cloud server...")
        response = requests.post(
            f"{base_url}/process-document", 
            json=test_data, 
            timeout=60,  # Tăng timeout cho cloud
            headers={'Content-Type': 'application/json'}
        )
        
        print(f"\nProcess Document Status: {response.status_code}")
        
        if response.status_code == 200:
            print("✓ SUCCESS!")
            print(f"Response: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")
        else:
            print(f"❌ FAILED!")
            print(f"Error Response: {response.text}")
            
    except requests.exceptions.Timeout:
        print("❌ Request timeout - Cloud function might be cold starting")
    except Exception as e:
        print(f"❌ Cloud test failed: {e}")


if __name__ == "__main__":
    test_cloud_server() 



=== TESTING CLOUD SERVER ===
Health: 200 - {'status': 'OK', 'timestamp': '2025-06-26T09:42:11.698Z'}

=== TESTING DEBUG APPROVAL ===
Debug Approval Status: 200
✓ Debug approval endpoint working

=== TESTING PROCESS DOCUMENT ===
Sending request to cloud server...

Process Document Status: 200
✓ SUCCESS!
Response: {
  "success": true,
  "instanceCode": "D3481A6A-EF1A-4F78-A933-E08DE378676F",
  "message": "Đã tạo thành công đơn phê duyệt văn bản với 1 tài liệu đính kèm",
  "documentInfo": {
    "loaiVanBan": "Công văn nội bộ"
  },
  "uploadedCodes": [
    "DF04770E-40EF-4276-8ED1-74F44BB7B755"
  ],
  "documentCount": 1
}


In [22]:
import json

# JSON response từ API
response_data = {
    "body": {
        "code": 0,
        "data": {
            "user_list": [
                {
                    "status": {
                        "is_exited": False,
                        "is_frozen": False,
                        "is_resigned": False,
                        "is_unjoin": False,
                        "is_activated": True
                    },
                    "user_id": "ou_7724fc05f615b0dfb76403574ad717e7",
                    "email": "le.syduc.993@gmail.com"
                }
            ]
        },
        "msg": "success"
    },
    "headers": {
        # ... headers data
    },
    "status_code": 200
}

# Cách 1: Truy cập trực tiếp
try:
    user_id = response_data['body']['data']['user_list'][0]['user_id']
    print(f"User ID: {user_id}")
except (KeyError, IndexError) as e:
    print(f"Lỗi khi lấy user_id: {e}")

# Cách 2: Kiểm tra an toàn hơn
def get_user_id(response):
    try:
        body = response.get('body', {})
        data = body.get('data', {})
        user_list = data.get('user_list', [])
        
        if user_list and len(user_list) > 0:
            return user_list[0].get('user_id')
        else:
            return None
    except Exception as e:
        print(f"Lỗi: {e}")
        return None

# Sử dụng
user_id = get_user_id(response_data)
if user_id:
    print(f"User ID tìm thấy: {user_id}")
else:
    print("Không tìm thấy user_id")

# Cách 3: Lấy tất cả thông tin user
def get_all_users_info(response):
    try:
        user_list = response['body']['data']['user_list']
        users_info = []
        
        for user in user_list:
            user_info = {
                'user_id': user.get('user_id'),
                'email': user.get('email'),
                'is_activated': user.get('status', {}).get('is_activated', False)
            }
            users_info.append(user_info)
        
        return users_info
    except Exception as e:
        print(f"Lỗi khi lấy thông tin users: {e}")
        return []

# Sử dụng
all_users = get_all_users_info(response_data)
for user in all_users:
    print(f"User ID: {user['user_id']}, Email: {user['email']}, Activated: {user['is_activated']}")


User ID: ou_7724fc05f615b0dfb76403574ad717e7
User ID tìm thấy: ou_7724fc05f615b0dfb76403574ad717e7
User ID: ou_7724fc05f615b0dfb76403574ad717e7, Email: le.syduc.993@gmail.com, Activated: True
